## Тесты

In [ ]:
from src.levenstein_rec import levenstein_rec
from src.levenstein_iter import levenstein_iter
from src.damerau_levenstein import damerau_levenstein

cases = [
    ("lorem ipsum", "dolor sit amet"),
    ("lorem ipsum", ""),
    ("", "consectetur"),
    ("LaTeX", "latex"),
    ("latex", "altex"),
    ("latex", "laTxe"),
    ("арбуз", "автобус")
]

results = [
    (10, 10),
    (11, 11),
    (11, 11),
    (3, 3),
    (2, 1),
    (3, 2),
    (4, 4)
]

fails = 0
for i in range(len(cases)):
    if levenstein_rec(
        cases[i][0],
        cases[i][1]
    ) != results[i][0]:
        print(f"failed recursive levenstein test with strings {
              cases[i][0]} and {cases[i][1]}")
        fails += 1
        continue

    if levenstein_iter(
        cases[i][0],
        cases[i][1]
    ) != results[i][0]:
        print(f"failed iter levenstein test with strings {
              cases[i][0]} and {cases[i][1]}")
        fails += 1
        continue

    if damerau_levenstein(
        cases[i][0],
        cases[i][1]
    ) != results[i][1]:
        print(f"failed damerau levenstein test with strings {
              cases[i][0]} and {cases[i][1]}")
        fails += 1
        continue

print(f"{len(cases) - fails}/{len(cases)} tests passed")

In [ ]:
from src.levenstein_rec import levenstein_rec
from src.levenstein_iter import levenstein_iter
from src.damerau_levenstein import damerau_levenstein

s1 = "abcdef"
s2 = "acbedf"

# a  b->''  c   d->b    e   ''->d   f
# M  D      M   R       M   A       M
# 0  1      0   1       0   1       0 = 3

print(f"Рекурсивное расстояние Левенштейна: {levenstein_rec(s1, s2)}")
print(f"Итерационное расстояние Левенштейна: {levenstein_iter(s1, s2)}")
print(f"Расстояние Дамерау-Левенштейна: {damerau_levenstein(s1, s2)}")

# a  b<->c d<->e f
# M    S     S   M
# 0    1     1   0 = 2

## Утилиты и константы

In [3]:
import string
from faker import Faker
fake = Faker()


def random_string(len: int) -> str:
    return ''.join(fake.random_choices(string.ascii_letters, len))

In [4]:
LEN_MIN = 1
LEN_MAX = 50
LEN_STEP = 1
MAX_REC_LENGTH = 10

TEST_COUNT = 50

## Замеры времени

In [ ]:
import time

times: list[tuple[int, int, int, int]] = []

for len in range(LEN_MIN, LEN_MAX+1, LEN_STEP):
    time_rec = 0
    time_iter = 0
    time_damerau = 0
    print(f"Processing {len}...              ", end="\r")

    for _ in range(TEST_COUNT):
        s1 = random_string(len)
        s2 = random_string(len)

        if len <= MAX_REC_LENGTH:
            start = time.process_time_ns()
            levenstein_rec(s1, s2)
            stop = time.process_time_ns()

            time_rec += (stop - start)
        else:
            time_rec = -1

        start = time.process_time_ns()
        levenstein_iter(s1, s2)
        stop = time.process_time_ns()

        time_iter += (stop - start)

        start = time.process_time_ns()
        damerau_levenstein(s1, s2)
        stop = time.process_time_ns()

        time_damerau += (stop - start)

    times.append((len, time_rec, time_iter, time_damerau))

print(times)

In [ ]:
import pandas as pd

if TEST_COUNT != 1:
    frame = pd.DataFrame(
        {
            "Длина": [record[0] for record in times],
            "Рек. алгоритм (нс)": pd.Series([record[1] for record in times]),
            "Итер. алгоритм (нс)": pd.Series([record[2] for record in times]),
            "Дамерау-Левенштейна (нс)": pd.Series([record[3] for record in times]),
        }
    )

    frame = frame.set_index("Длина")

    print(f"Таблица зависимости времени обработки строк от длины строк {
        TEST_COUNT} прогонах")
    print(frame)

In [ ]:
import pandas as pd

frame = pd.DataFrame(
    {
        "Длина": [record[0] for record in times],
        "Рек. алгоритм (нс)": pd.Series([record[1] // TEST_COUNT for record in times]),
        "Итер. алгоритм (нс)": pd.Series([record[2] // TEST_COUNT for record in times]),
        "Дамерау-Левенштейна (нс)": pd.Series([record[3] // TEST_COUNT for record in times]),
    }
)

frame = frame.set_index("Длина")

print(f"Таблица зависимости времени обработки строк от длины строк")

print(frame)

In [ ]:
import matplotlib.pyplot as plt

xrec = []
yrec = []

for record in times:
    if record[0] > MAX_REC_LENGTH:
        break

    xrec.append(record[0])
    yrec.append(record[1] // TEST_COUNT // 1000)

plt.plot(xrec, yrec, label="Рекурсивный алгоритм Левенштейна")
plt.plot([record[0] for record in times], [record[2] // TEST_COUNT // 1000
         for record in times], label="Итерационный алгоритм Левенштейна")
plt.plot([record[0] for record in times], [record[3] // TEST_COUNT // 1000
         for record in times], label="Итерационный алгоритм Дамерау-Левенштейна")

plt.xlabel("Длина строк")
plt.ylabel("Время обработки (мкс)")

plt.xlim(0, max([record[0] for record in times]))
plt.ylim(0, max(yrec))

plt.legend()

plt.tight_layout()
plt.savefig("./docs/img/time_graph.svg")
plt.show()

plt.plot([record[0] for record in times], [record[2] // TEST_COUNT // 1000
         for record in times], label="Итерационный алгоритм Левенштейна")
plt.plot([record[0] for record in times], [record[3] // TEST_COUNT // 1000
         for record in times], label="Итерационный алгоритм Дамерау-Левенштейна")

plt.xlim(0, max([record[0] for record in times]))
plt.ylim(0, max([record[3] // TEST_COUNT // 1000
         for record in times]))

plt.xlabel("Длина строк")
plt.ylabel("Время обработки (мкс)")

plt.legend()
plt.savefig("./docs/img/time_graph_norec.svg")
plt.show()

## Замеры памяти

In [ ]:
import tracemalloc

mem: list[tuple[int, int, int]] = []

for len in range(LEN_MIN, MAX_REC_LENGTH + 1):
    mem_rec = 0
    mem_iter = 0
    print(f"Processing {len}...              ", end="\r")

    for _ in range(TEST_COUNT):
        s1 = random_string(len)
        s2 = random_string(len)

        tracemalloc.start()
        levenstein_rec(s1, s2)
        mem_rec += tracemalloc.get_traced_memory()[1]
        tracemalloc.stop()

        tracemalloc.start()
        levenstein_iter(s1, s2)
        mem_iter += tracemalloc.get_traced_memory()[1]
        tracemalloc.stop()

    mem.append((len, mem_rec, mem_iter))

In [ ]:
import pandas as pd

frame = pd.DataFrame(
    {
        "Длина": [record[0] for record in mem],
        "Рек. алгоритм (Байт)": pd.Series([record[1] // TEST_COUNT for record in mem]),
        "Итер. алгоритм (Байт)": pd.Series([record[2] // TEST_COUNT for record in mem]),
    }
)

frame = frame.set_index("Длина")

print(f"Таблица зависимости объёма использованной при обработке строк памяти от их длины")
print(frame)

In [ ]:
import matplotlib.pyplot as plt

plt.xlabel("Длина строк")
plt.ylabel("Объём использованной памяти (Байт)")

plt.plot([record[0] for record in mem], [record[1] //
         TEST_COUNT for record in mem], label="Рекурсивный алгоритм Левенштейна")
plt.plot([record[0] for record in mem], [record[2] //
         TEST_COUNT for record in mem], label="Итерационный алгоритм Левенштейна")

plt.legend()

plt.tight_layout()

plt.savefig("./docs/img/mem_graph.svg")
plt.show()